In [9]:
%run runaway_functionsv3

In [5]:
workclusters

['ASCC_107',
 'ASCC_114',
 'ASCC_127',
 'ASCC_13',
 'ASCC_16',
 'ASCC_19',
 'ASCC_21',
 'ASCC_32',
 'ASCC_67',
 'ASCC_9',
 'Alessi_20',
 'Alessi_43',
 'Alessi_Teutsch_5',
 'Antalova_2',
 'Archinal_1',
 'Aveni_Hunter_1',
 'BDSB91',
 'BDSB93',
 'BDSB96',
 'BH_121',
 'BH_200',
 'BH_205',
 'BH_217',
 'BH_221',
 'BH_245',
 'BH_54',
 'BH_56',
 'BH_87',
 'BH_92',
 'Barkhatova_1',
 'Basel_18',
 'Basel_8',
 'Berkeley_11',
 'Berkeley_15',
 'Berkeley_47',
 'Berkeley_62',
 'Berkeley_65',
 'Berkeley_7',
 'Berkeley_79',
 'Berkeley_86',
 'Berkeley_87',
 'Berkeley_97',
 'Bica_2',
 'Biurakan_2',
 'Bochum_10',
 'Bochum_11',
 'Bochum_13',
 'COIN-Gaia_16',
 'COIN-Gaia_21',
 'COIN-Gaia_35',
 'COIN-Gaia_41',
 'Collinder_104',
 'Collinder_106',
 'Collinder_107',
 'Collinder_132',
 'Collinder_197',
 'Collinder_205',
 'Collinder_272',
 'Collinder_419',
 'Collinder_421',
 'Collinder_69',
 'Collinder_95',
 'Czernik_1',
 'Czernik_31',
 'Czernik_41',
 'Czernik_6',
 'Czernik_8',
 'Dias_1',
 'Dias_5',
 'Dolidze_16',

In [29]:
for cluster in workclusters[:]:
    print("-"*50)
    print(cluster)
    print("-"*50)
    Cluster(cluster).members()

--------------------------------------------------
ASCC_107
--------------------------------------------------
73 out of 92 dias members found in search region
Changed N          59.00 --> 46.00
Changed Plx        1.12 --> 1.14
Changed e_Plx      0.05 --> 0.04
Changed Dist       864.00 --> 853.38
Changed e_Dist     30.00 --> 27.52
Changed pmRA       -0.14 --> -0.19
Changed e_pmRA     0.14 --> 0.11
Changed pmDE       -5.16 --> -5.16
Changed e_pmDE     0.14 --> 0.11
Changed RV         -- --> -4.34
Changed e_RV       -- --> 3.33
Changed NRV        -- --> 28.00
--------------------------------------------------
ASCC_114
--------------------------------------------------
167 out of 208 dias members found in search region
Changed N          149.00 --> 110.00
Changed Plx        1.06 --> 1.07
Changed e_Plx      0.04 --> 0.03
Changed Dist       911.00 --> 917.03
Changed e_Dist     12.00 --> 22.90
Changed pmRA       -3.75 --> -3.72
Changed e_pmRA     0.21 --> 0.10
Changed pmDE       -3.43 --> -3

In [26]:
dias2021 = Table.read('dias2021.tsv', format='ascii.ecsv')
dias2021[dias2021['Cluster']=='Basel_8']

Cluster,RA_ICRS,DE_ICRS,r50,N,pmRA,e_pmRA,pmDE,e_pmDE,Plx,e_Plx,RV,e_RV,NRV,Dist,e_Dist,logage,e_logage,__Fe_H_,e__Fe_H_,Av,e_Av,FileName,SimbadName,_RA.icrs,_DE.icrs
,deg,deg,deg,,mas / yr,mas / yr,mas / yr,mas / yr,mas,mas,km / s,km / s,,pc,pc,log(yr),log(yr),,,mag,mag,,,deg,deg
str17,float64,float64,float32,int16,float32,float32,float32,float32,float32,float32,float64,float32,int16,int16,int16,float32,float32,float32,float32,float32,float32,str31,str31,float64,float64
Basel_8,98.5582,8.0381,0.299,31,-0.134,0.066,-2.400,0.140,0.623,0.036,--,--,--,1456,54,7.671,0.236,0.028,0.064,1.083,0.079,clusters1/Basel_8.dat,Cl Basel 8,98.5582,8.0381


In [65]:
cl = Cluster("Basel_8")
np.count_nonzero(~cl.stars_in_region()['RV'].mask)


375

In [22]:

%run runaway_functionsv3
cl = Cluster("ASCC_107")
cl.stars_in_region()
t = cl.mymembers
t = cl.stars_in_region()
t = find_cluster(cl.stars_in_region(), sigma=1.5)

np.mean(t['RV']),np.sqrt(np.sum(t['e_RV'])**2/(len(t))**2),(np.count_nonzero(~(t['RV'].mask)))

(-5.870175438596491, 3.336015055950423, 57)

In [53]:
cl.all

Cluster,RA_ICRS,DE_ICRS,r50,N,pmRA,e_pmRA,pmDE,e_pmDE,Plx,e_Plx,RV,e_RV,NRV,Dist,e_Dist,logage,e_logage,__Fe_H_,e__Fe_H_,Av,e_Av,FileName,SimbadName,_RA.icrs,_DE.icrs
,deg,deg,deg,,mas / yr,mas / yr,mas / yr,mas / yr,mas,mas,km / s,km / s,,pc,pc,log(yr),log(yr),,,mag,mag,,,deg,deg
str17,float64,float64,float32,int16,float32,float32,float32,float32,float32,float32,float64,float32,int16,int16,int16,float32,float32,float32,float32,float32,float32,str31,str31,float64,float64
Basel_8,98.5582,8.0381,0.299,8,-0.190,0.225,-2.183,0.225,0.653,0.027,26.437,1.331,35,1450,49,7.671,0.236,0.028,0.064,1.083,0.079,clusters1/Basel_8.dat,Cl Basel 8,98.5582,8.0381


In [18]:
cl.kinematic_cluster

RA_ICRS_1,DE_ICRS_1,e_RA_ICRS,e_DE_ICRS,_r_1,HIP,TYC2,Source,rgeo,Plx,e_Plx,pmRA,pmDE,e_pmRA,e_pmDE,RUWE,Teff,logg,Gmag,BP-RP,BPmag,RPmag,RV,e_RV,b_rgeo,B_rgeo,FG,e_FG,FBP,e_FBP,FRP,e_FRP,RAVE5,RAVE6,e_Gmag,e_BPmag,e_RPmag,e_BP-RP,SkyCoord,rmRA,rmDE,e_rmRA,e_rmDE,rRV,e_rRV,Temp. Est
deg,deg,mas,mas,,,,,pc,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,,K,log(cm.s**-2),mag,mag,mag,mag,km / s,km / s,pc,pc,,,,,,,,,,,,,"deg,deg,pc",mas / yr,mas / yr,mas / yr,mas / yr,km / s,km / s,
float64,float64,float64,float64,float64,int32,str11,int64,float64,float64,float32,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float32,float64,float64,float64,float32,float32,float32,float32,float32,str1,str1,float64,float32,float32,float32,SkyCoord,float64,float64,float32,float32,float64,float32,int64
297.17084324992,21.98476752483,0.0182,0.0199,1.4220,97471,1627-332-1,1827560055337367296,905.13012700,1.0861,0.0281,0.115,-5.226,0.024,0.026,0.972,18863.6,3.4733,8.152393,0.281548,8.234911,7.953362,--,--,883.81189000,929.28149400,10327361.03773,3664,6.942e+06,5500,5.222e+06,3230,--,--,0.002782,0.00292,0.003839,0.006758,"297.17084324992,21.98476752483,905.130127",0.30682609021663665,-0.0668694267272949,0.13751926,0.13951926,--,--,0
297.11085896723,21.96246797013,0.0365,0.0388,3.9200,--,1627-11-1,1827554282901324032,929.58429000,1.0677,0.0542,-0.429,-5.290,0.050,0.050,2.939,16531.8,3.8265,9.317932,0.302243,9.410587,9.108344,--,--,877.12780800,979.60638400,3529993.54319,998.6,2.351e+06,1625,1.802e+06,855.7,--,--,0.002772,0.002889,0.003814,0.006704,"297.11085896723,21.96246797013,929.58429",-0.23717390978336333,-0.13086942672729496,0.16351926,0.16351926,--,--,0
297.10135591695,21.91424698051,0.0116,0.0137,6.5227,--,1627-790-1,1827553496963958144,859.21020500,1.1238,0.0186,0.014,-5.148,0.016,0.017,0.946,11621.6,3.9140,11.037607,0.443225,11.193063,10.749838,-27.00,12.06,846.32116700,872.17327900,724275.30768,193.6,4.552e+05,219.1,3.974e+05,99.14,--,--,0.002771,0.002839,0.003789,0.006628,"297.10135591695,21.91424698051,859.210205",0.20582609021663667,0.01113057327270539,0.12951925,0.13051926,-21.13,15.40,0
296.82257860134,22.11386915659,0.0114,0.0121,19.9478,--,1627-1658-1,1827572321763952128,916.31494100,1.0526,0.0156,1.317,-4.259,0.015,0.016,0.907,10818.1,3.8842,11.066307,0.717512,11.336892,10.619380,-4.68,6.24,905.36102300,927.64782700,705381.56671,119.3,3.987e+05,209.6,4.481e+05,102.3,--,--,0.002761,0.002848,0.003787,0.006635,"296.82257860134,22.11386915659,916.314941",1.5088260902166366,0.9001305732727047,0.12851925,0.12951925,1.19,9.58,0
297.20914586998,21.78557955990,0.0170,0.0200,13.5450,--,1627-970-1,1827501785569485952,855.90472400,1.1426,0.0254,-1.889,-4.495,0.022,0.026,1.306,5062.0,2.8506,11.113397,1.603019,11.871234,10.268215,-1.71,0.23,836.92456100,874.39117400,675442.14013,207.2,2.437e+05,130.7,6.193e+05,273.6,--,--,0.002775,0.00285,0.00381,0.00666,"297.20914586998,21.7855795599,855.904724",-1.6971739097833634,0.664130573272705,0.13551927,0.13951926,4.16,3.57,0
296.99981774609,21.96166082885,0.0108,0.0119,9.4427,--,1627-1347-1,1827566484946171136,901.64007600,1.0658,0.0166,-0.200,-5.188,0.014,0.015,0.871,11839.5,4.0590,11.115017,0.482429,11.287463,10.805035,--,--,887.90368700,915.95544400,674435.06419,158.4,4.173e+05,205.7,3.777e+05,107.8,--,--,0.002767,0.002841,0.003792,0.006633,"296.99981774609,21.96166082885,901.640076",-0.008173909783363353,-0.028869426727294645,0.12751926,0.12851925,--,--,0
297.23011232398,21.89114185616,0.0092,0.0120,7.9152,--,1627-880-1,1827555730347495936,855.31518600,1.1190,0.0149,-0.156,-5.183,0.013,0.016,0.867,--,--,11.681152,0.626036,11.914623,11.288588,--,--,843.56378200,868.81396500,400392.32588,70.59,2.342e+05,122.1,2.42e+05,64.14,--,--,0.002762,0.002847,0.00379,0.006637,"297.23011232398,21.89114185616,855.315186",0.03582609021663666,-0.02386942672729475,0.12651926,0.12951925,--,--,0
297.07444806332,22.00880815725,0.0086,0.0086,4.8884,--,1627-149-1,1827555421109423360,

In [17]:
%run runaway_functionsv3
%matplotlib qt
cl = Cluster("ASCC_107")
# cir = find_cluster(cl.stars_in_region())


cl.plot_traceback_clean(star_tables=[cl.kinematic_cluster, 
                                    #  cir2
                                     ])

qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


In [45]:
cir

RA_ICRS_1,DE_ICRS_1,e_RA_ICRS,e_DE_ICRS,_r_1,HIP,TYC2,Source,rgeo,Plx,e_Plx,pmRA,pmDE,e_pmRA,e_pmDE,RUWE,Teff,logg,Gmag,BP-RP,BPmag,RPmag,RV,e_RV,b_rgeo,B_rgeo,FG,e_FG,FBP,e_FBP,FRP,e_FRP,RAVE5,RAVE6,e_Gmag,e_BPmag,e_RPmag,e_BP-RP,SkyCoord,rmRA,rmDE,e_rmRA,e_rmDE,rRV,e_rRV,Temp. Est
deg,deg,mas,mas,,,,,pc,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,,K,log(cm.s**-2),mag,mag,mag,mag,km / s,km / s,pc,pc,,,,,,,,,,,,,"deg,deg,pc",mas / yr,mas / yr,mas / yr,mas / yr,km / s,km / s,
float64,float64,float64,float64,float64,int32,str11,int64,float64,float64,float32,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float32,float64,float64,float64,float32,float32,float32,float32,float32,str1,str1,float64,float32,float32,float32,SkyCoord,float64,float64,float32,float32,float64,float32,int64
339.84392032630,59.00715291679,0.0293,0.0308,0.2402,--,3995-625-1,2008383302396655488,2630.22485000,0.3652,0.0363,-2.873,-2.301,0.038,0.038,3.035,18135.5,3.3302,9.473729,0.937676,9.843719,8.906042,--,--,2393.19263000,2940.37549000,3058125.18769,951.5,1.577e+06,1154,2.171e+06,1124,--,--,0.002776,0.002901,0.003821,0.006722,"339.8439203263,59.00715291679,2630.22485",-0.1396428089141848,-0.41682148075103775,0.13697915,0.13697915,--,--,0
339.82384023875,59.01681945374,0.0097,0.0105,1.0890,--,3995-1075-1,2008430237808841600,2657.56860000,0.3583,0.0125,-2.747,-1.911,0.013,0.013,0.986,--,--,9.840384,0.868947,10.178231,9.309284,--,--,2587.60059000,2734.51587000,2181691.83228,1048,1.159e+06,1141,1.498e+06,1215,--,--,0.002804,0.002988,0.003881,0.006869,"339.82384023875,59.01681945374,2657.5686",-0.013642808914184457,-0.02682148075103763,0.11197916,0.11197916,--,--,0
340.10268328507,58.97390297864,0.0097,0.0107,8.0278,--,3996-1228-1,2008380794144211712,2570.86328000,0.3525,0.0122,-2.679,-1.930,0.013,0.014,0.903,--,--,11.482301,1.201067,11.985507,10.784440,--,--,2498.03125000,2653.16870000,480868.62054,798.2,2.194e+05,725.5,3.849e+05,2491,--,--,0.003292,0.004547,0.007978,0.01253,"340.10268328507,58.97390297864,2570.86328",0.054357191085815604,-0.045821480751037535,0.11197916,0.11297915,--,--,0
339.82166004461,59.00297803329,0.0089,0.0097,0.8784,--,--,2008430203449108224,2525.15747000,0.3648,0.0114,-2.953,-1.724,0.012,0.012,0.988,--,--,13.262117,0.961862,13.648018,12.686156,--,--,2459.89844000,2592.30542000,93346.90312,21.67,4.745e+04,60.24,6.679e+04,44.71,--,--,0.002767,0.003112,0.003849,0.006961,"339.82166004461,59.00297803329,2525.15747",-0.21964280891418442,0.16017851924896243,0.110979155,0.110979155,--,--,0
340.13166575232,58.80702105155,0.0325,0.0335,14.7229,--,--,2008366363053930112,2452.92578000,0.3907,0.0404,-2.667,-1.648,0.044,0.042,2.897,14207.5,3.6701,13.919951,1.094469,14.357768,13.263299,--,--,2208.02368000,2667.27612000,50929.10304,22.21,2.468e+04,22.44,3.925e+04,18.26,--,--,0.002796,0.00296,0.003813,0.006773,"340.13166575232,58.80702105155,2452.92578",0.06635719108581561,0.2361785192489625,0.14297915,0.14097916,--,--,0
340.05588361488,58.81770320410,0.0127,0.0133,12.9026,--,--,2008366981529199616,2631.17554000,0.3528,0.0158,-2.707,-2.076,0.017,0.016,1.080,9669.0,3.6676,14.161031,0.952089,14.543823,13.591734,--,--,2517.38721000,2756.26270000,40788.17956,8.078,2.079e+04,16.83,2.9e+04,14.29,--,--,0.002764,0.002925,0.003817,0.006742,"340.05588361488,58.8177032041,2631.17554",0.02635719108581558,-0.19182148075103767,0.11597915,0.114979155,--,--,0
339.84074866849,59.00514934930,0.0134,0.0139,0.2851,--,--,2008383302404812544,2604.80005000,0.3569,0.0163,-2.626,-1.913,0.018,0.017,1.124,11638.8,3.8049,14.199184,1.058014,14.629308,13.571294,--,--,2466.69116000,2720.19312000,39379.74224,11.71,1.922e+04,51.21,2.956e+04,27.21,--,--,0.002774,0.00402,0.003909,0.007929,"339.84074866849,59.0051493493,2604.80005",0.10735719108581554,-0.02882148075103763,0.11697915,0.11597915,--,--,0
339.85983016055,59.00845000045,0.0134,0.0137,0.3836,--,--,2008384779873564032,2623.42310000,0.3563,0.0168,-2.780,-1.834,0.017,0.016,1.021,10857.

In [36]:
%run runaway_functionsv3
%matplotlib qt
cl = Cluster("Berkeley_97")
cld = ClusterDias("Berkeley_97")
isochrone1 = Isochrone(cld)
isochrone2 = Isochrone(cl, Av=3, logage=7)
plot_cmd(cl, isochrones=[isochrone1,isochrone2])
# plot_cmd(cld)



In [48]:
source = 2008384504995669248
object = f"gaia dr3 {source}"
print(object, type(object))
Simbad.query_object("gaia dr3 2008383302396655488")

gaia dr3 2008384504995669248 <class 'str'>


MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,COO_BIBCODE,SCRIPT_NUMBER_ID
,"""h:m:s""","""d:m:s""",,,mas,mas,deg,,,,
object,str13,str13,int16,int16,float32,float32,int16,str1,str1,object,int32
HD 240016,22 39 22.5468,+59 00 25.787,14,14,0.029,0.031,90,A,O,2020yCat.1350....0G,1


In [45]:
cl.runaways()

RA_ICRS_1,DE_ICRS_1,rgeo,Teff,Temp. Est,e_RA_ICRS,e_DE_ICRS,_r_1,HIP,TYC2,Source,Plx,e_Plx,pmRA,pmDE,e_pmRA,e_pmDE,RUWE,Gmag,BP-RP,BPmag,RPmag,b_rgeo,B_rgeo,e_Gmag,e_BPmag,e_RPmag,e_BP-RP,SkyCoord,rmRA,rmDE,v_pec,logg,RV,e_RV,FG,e_FG,FBP,e_FBP,FRP,e_FRP,RAVE5,RAVE6
deg,deg,pc,K,K,mas,mas,,,,,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,,mag,mag,mag,mag,pc,pc,,,,,"deg,deg,pc",mas / yr,mas / yr,km / s,log(cm.s**-2),km / s,km / s,,,,,,,,
float64,float64,float64,float64,float64,float64,float64,float64,int32,str11,int64,float64,float32,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,SkyCoord,float64,float64,float64,float64,float64,float32,float64,float32,float32,float32,float32,float32,str1,str1
339.89581711423,58.99547917199,2620.36230000,9168.0,10920.0,0.0202,0.0197,1.5205,--,--,2008384504995669248,0.3560,0.0241,-0.379,-1.352,0.025,0.024,1.086,15.384143,1.182070,15.883185,14.701116,2461.53345000,2759.61401000,0.00277,0.003852,0.004229,0.008081,"339.89581711423,58.99547917199,2620.3623",2.3543571910858154,0.5321785192489623,29.98008203419036,3.7061,--,--,13221.76503,3.482,6055,14.81,1.044e+04,18.25,--,--


In [44]:
for runaway in cl.runaways():
    source = runaway['Source']
    print(source)
    
    t = Simbad.query_object(f"gaia dr3 {source}")['MAIN_ID']
    print(t)

2008384504995669248


/usr/lib/python3/dist-packages/astroquery/simbad/core.py:135: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): Identifier not found in the database : Gaia DR3 2008384504995669248
  warnings.warn("Warning: The script line number %i raised "


TypeError: 'NoneType' object is not subscriptable

In [41]:
from astroquery.simbad import Simbad
t = Simbad.query_object("gaia dr3 2008384504995669248")
t

/usr/lib/python3/dist-packages/astroquery/simbad/core.py:135: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): Identifier not found in the database : Gaia DR3 2008384504995669248
  warnings.warn("Warning: The script line number %i raised "


In [13]:
dias2021 = Table.read("dias2021.tsv", format="ascii.ecsv")
maskplx = dias2021['Plx'] > 0.3
maskage = dias2021['logage'] < 7.7
workclusters = []
for clustername in dias2021[maskplx & maskage][:]['Cluster']:
    if clustername not in ['ASCC_79','BH_164','BH_23','Collinder_135','Collinder_140','Gulliver_9','IC_2391','IC_2602','Mamajek_1','Platais_8','UPK_535','UPK_606','UPK_640','Berkeley_59','COIN-Gaia_37','Ivanov_4','LP_1937','Sigma_Ori','UBC_632']:
        workclusters.append(clustername)
        # print(clustername)
        # cl = Cluster(clustername)
        # cl.prepare_trace()
# cl.stars_in_region()
# cl.stars_in_region()

In [ ]:
workclusters_mask = [cluster in workclusters for cluster in dias2021['Cluster']]
workclusterstable = dias2021[workclusters_mask]

rvexists = workclusterstable['RV'].mask
nrvgood = workclusterstable['NRV'] > 4

workclusterstable[~rvexists & nrvgood]

In [ ]:
def runaways_from(clustername):
    cl = Cluster(clustername)
    fs4giesler = cl.fast_stars_in_region()
    outputs = os.listdir(f"/home/surodeep/suro_aiu/traceback/cluster_runaway/{cl.name}/runaways/")
    linenos = []
    for output in outputs:
        #print(output)
        if 'run' in output:
            linenos.append(int(output.split("+")[1].replace(".out","")))
    linenos.sort()
    # print(linenos)
    i=np.array(linenos)-3
    def source_of(lineno, input_table):
        return input_table[lineno-2]['Source']
    return fs4giesler[i]
for cluster in workclusters:
    t = runaways_from(cluster)[runaways_from(cluster)['Teff'].argmax()]['Teff']
    
    if t>10000:
        print(cluster, t,runaways_from(cluster)[runaways_from(cluster)['Teff'].argmax()]['Source'])